In [8]:
# _dir = '/data/female-amb/glider_25K_impute/'
# all_imputed_files = glob.glob(f'{_dir}/*/*/group*/impute/25K/group*/chunk*/*.cool')
# cell_id = [path.split('/')[-1].split('.')[0] for path in all_imputed_files]
# cell_teble = pd.DataFrame({'cell_id':cell_id,'file_paths':all_imputed_files})
# cell_teble.to_csv('All.25K.ImputedCool.csv')

In [1]:
import glob
import pandas as pd
import joblib
import subprocess
import pathlib

In [2]:
ct = 'Oligo_NN'

In [3]:
use_cts = joblib.load('/data/metadata/m3c_use_cts')
use_cts = [ct.replace(' ','_').replace('/','') for ct in use_cts]

In [4]:
meta = pd.read_csv('/data/metadata/240104_m3C_META.csv', index_col = 0)
meta['AgingMajorType'] = meta['AgingMajorType'].apply(lambda x: x.replace(' ','_').replace('/',''))
meta = meta[meta['AgingMajorType'].isin(use_cts)]
meta['AgingMajorType.Age'] = meta['AgingMajorType'] + '.' + meta['Age']

In [5]:
cell_table = pd.read_csv('All.25K.ImputedCool.csv', index_col = 'cell_id')[['file_paths']]
cell_table = pd.concat([cell_table, meta[['AgingMajorType','Age','AgingMajorType.Age']]], axis = 1)
cell_table = cell_table.dropna()
cell_table.head()

,file_paths,AgingMajorType,Age,AgingMajorType.Age
AMB_220426_8wk_8E_9H_8J_9J_1_P1-1-K15-A1,/data/female-amb/glider_25K_impute/0/0/group43...,Sst_Gaba,8wk,Sst_Gaba.8wk
AMB_220426_8wk_8E_9H_8J_9J_1_P1-1-K15-A13,/data/female-amb/glider_25K_impute/0/0/group43...,Astro-TE_NN,8wk,Astro-TE_NN.8wk
AMB_220426_8wk_8E_9H_8J_9J_1_P1-1-K15-A14,/data/female-amb/glider_25K_impute/0/0/group43...,Astro-TE_NN,8wk,Astro-TE_NN.8wk
AMB_220426_8wk_8E_9H_8J_9J_1_P1-1-K15-B1,/data/female-amb/glider_25K_impute/0/0/group43...,Astro-TE_NN,8wk,Astro-TE_NN.8wk
AMB_220426_8wk_8E_9H_8J_9J_1_P1-1-K15-B13,/data/female-amb/glider_25K_impute/0/0/group43...,DG_Glut,8wk,DG_Glut.8wk


In [6]:
use_cell_table = cell_table[cell_table['AgingMajorType']== ct]

In [7]:
all_groups = []
for ct_age, tmpdf in use_cell_table.groupby('AgingMajorType.Age'):
    tmpdf['cell_id'] = tmpdf.index
    tmpdf = tmpdf[['cell_id','file_paths']]
    tmpdf = tmpdf.sample(300)
    tmpdf.to_csv(f"cell_table/{ct_age}.tsv", sep = '\t', header = None, index = None)
    all_groups.append(ct_age)

In [2]:
#output_dir = 'dataset/domain'
output_dir = 'test'
p = pathlib.Path(output_dir)
p.mkdir(parents=True, exist_ok=True)

In [3]:
cpu = 40

In [5]:
with open("boundary.sh", 'w') as file:
    for group in all_groups:
        cell_table_path = f'cool_table/{group}.tsv'
        output_prefix = f'{output_dir}/{group}'
        command = f"hicluster domain \
--cell_table_path {cell_table_path} \
--output_prefix {output_prefix} \
--resolution 25000 \
--window_size 10 \
--cpu {cpu}"
        file.write(f"{command}")   
        file.write("\n")   

NameError: name 'all_groups' is not defined

In [15]:
#subprocess.run(command, shell = True)